In [2]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
load_dotenv()
import os
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model_name = "openai/gpt-oss-120b",
)

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({"topic":"Pizza"},config=config1)

{'topic': 'Pizza',
 'joke': 'Why did the pizza apply for a job?\n\nBecause it kneaded the dough, wanted a slice of the action, and heard the workplace had great “toppings” for career growth! 🍕😄',
 'explanation': '**The joke:**  \n*Why did the pizza apply for a job?*  \n*Because it kneaded the dough, wanted a slice of the action, and heard the workplace had great “toppings” for career growth! 🍕😄*\n\n**Why it’s funny – a breakdown of the wordplay**\n\n| Part of the punchline | What it literally means (pizza‑related) | What it means in a normal work‑context (the “real” meaning) | Why the double meaning is amusing |\n|-----------------------|------------------------------------------|------------------------------------------------------------|-----------------------------------|\n| **“kneaded the dough”** | Pizza dough is *kneaded* (pressed, folded, and stretched) to develop gluten before baking. | “Needed the dough” = needed money or a paycheck. | The homophone “kneaded” (the baking acti

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Pizza', 'joke': 'Why did the pizza apply for a job?\n\nBecause it kneaded the dough, wanted a slice of the action, and heard the workplace had great “toppings” for career growth! 🍕😄', 'explanation': '**The joke:**  \n*Why did the pizza apply for a job?*  \n*Because it kneaded the dough, wanted a slice of the action, and heard the workplace had great “toppings” for career growth! 🍕😄*\n\n**Why it’s funny – a breakdown of the wordplay**\n\n| Part of the punchline | What it literally means (pizza‑related) | What it means in a normal work‑context (the “real” meaning) | Why the double meaning is amusing |\n|-----------------------|------------------------------------------|------------------------------------------------------------|-----------------------------------|\n| **“kneaded the dough”** | Pizza dough is *kneaded* (pressed, folded, and stretched) to develop gluten before baking. | “Needed the dough” = needed money or a paycheck. | The homophone “knead

In [10]:
config2 = {"configurable":{"thread_id":"2"}}
workflow.invoke({"topic":"Mango"},config=config2)

{'topic': 'Mango',
 'joke': 'Why did the mango bring a ladder to the party?  \n\nBecause it heard the punch‑line was going to be *a-peel*ing and wanted to reach the “top” of the jokes! 🍑😄',
 'explanation': '**What the joke is doing**\n\n1. **Set‑up:**  \n   *“Why did the mango bring a ladder to the party?”*  \n   This makes us picture a mango (a fruit) behaving like a person and doing something odd—bringing a ladder. It primes us to look for a reason that will involve a play on words.\n\n2. **Punch‑line:**  \n   *“Because it heard the punch‑line was going to be *a‑peel*ing and wanted to reach the ‘top’ of the jokes!”*  \n\n3. **How the wordplay works**\n\n| Element | Literal meaning | Pun / double meaning |\n|---------|----------------|----------------------|\n| **Mango** | A tropical fruit. | The joke treats the mango as a character that can think, hear, and act. |\n| **Ladder** | A tool for climbing to a higher place. | In the joke the mango uses the ladder to “reach” something that’

### Fault Tolerance

In [11]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [12]:
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str

In [13]:
def step1(state:CrashState):
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

In [14]:
def step2(state:CrashState):
    print("✅ Step 2 executed")
    time.sleep(5)
    return {"step2": "done", "input": state["input"]}

In [15]:
def step3(state:CrashState):
    print("✅ Step 3 executed")
    return {"step3": "done", "input": state["input"]}

In [16]:
graph = StateGraph(CrashState)

graph.add_node("step_1",step1)
graph.add_node("step_2",step2)
graph.add_node("step_3",step3)

graph.add_edge(START,"step_1")
graph.add_edge("step_1","step_2")
graph.add_edge("step_2","step_3")
graph.add_edge("step_3",END)

checkpointer = checkpointer
workflow = graph.compile(checkpointer=checkpointer)

In [19]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    workflow.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
✅ Step 2 executed
✅ Step 3 executed


In [20]:
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = workflow.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}


In [22]:
list(workflow.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f1067e6-c336-63ae-8003-c3d1d7677d09'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-10T12:45:47.614916+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f1067e6-c32e-6d30-8002-5b9fb003606f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f1067e6-c32e-6d30-8002-5b9fb003606f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-10T12:45:47.611883+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f1067e6-937c-691c-8001-f95b5c529206'}}, tasks=(PregelTask(id='8db11057-f2a0-cd5b